In [1]:
import os
import pickle
import re
import pprint
import numpy as np
import sympy as sp
import openturns as ot
# Display the final graph
import openturns.viewer as viewer
import matplotlib.pyplot as plt
import trimesh as tr
from importlib import reload
from functools import partial

from math import pi
from joblib import Parallel, delayed
from importlib import reload
from IPython.display import display, clear_output
from time import time
from sympy.printing import latex
from trimesh import viewer as trview
import sklearn

from scipy.optimize import OptimizeResult, minimize, basinhopping, differential_evolution, brute, shgo, check_grad, approx_fprime, fsolve, NonlinearConstraint, Bounds

import tqdm
import otaf

from gldpy import GLD

ot.Log.Show(ot.Log.NONE)
np.set_printoptions(suppress=True)
ar = np.array

# Notebook for the analysis of a system comprised of N + 2 parts, 2 plates with N = N1 x N2 holes, and N pins. 

### Defintion on global descriptive parameters

In [2]:
NX = 2 ## Number of holes on x axis
NY = 2 ## Number of holes on y axis
Dext = 20 ## Diameter of holes in mm
Dint = 19.8 ## Diameter of pins in mm
EH = 50 ## Distance between the hole axises
LB = 25 # Distance between border holes axis and edge.
hPlate = 30 #Height of the plates in mm
hPin = 60 #Height of the pins in mm

CIRCLE_RESOLUTION = 16 # NUmber of points to model the contour of the outer holes

### Defining and constructing the system data dictionary

The plates have NX * NY + 1 surfaces. The lower left point has coordinate 0,0,0

We only model the surfaces that are touching. 

In [3]:
N_PARTS = NX * NY * 2
LX = (NX - 1) * EH + 2*LB
LY = (NY - 1) * EH + 2*LB

contour_points = ar([[0,0,0],[LX,0,0],[LX,LY,0],[0,LY,0]])

R0 = ar([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
x_, y_, z_ = R0[0], R0[1], R0[2]

Frame1 = ar([z_,y_,-x_])
Frame2 = ar([-z_,y_,x_])

First we define the base part dictionaries for the upper and lower plate, without holes

In [4]:
system_data = {
    "PARTS" : {
        '0' : {
            "a" : {
                "FRAME": Frame1,
                "POINTS": {'A0' : ar([0,0,0]),
                           'A1' : ar([LX,0,0]),
                           'A2' : ar([LX,LY,0]),
                           'A3' : ar([0,LY,0]),
                        },
                "TYPE": "plane",
                "INTERACTIONS": ['P1a'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            }
        },
        '1' : {
            "a" : {
                "FRAME": Frame2,
                "POINTS": {'A0' : ar([0,0,0]),
                           'A1' : ar([LX,0,0]),
                           'A2' : ar([LX,LY,0]),
                           'A3' : ar([0,LY,0]),
                        },
                "TYPE": "plane",
                "INTERACTIONS": ['P0a'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            }
        }  
    },
    "LOOPS": {
        "COMPATIBILITY": {
        },
    },
    "GLOBAL_CONSTRAINTS": "3D",
}

Then we iterate over the pin dimensions NX and NY, and create the corresponding holes and pins. At the same time there is 1 loop per pin

In [5]:
alpha_gen = otaf.common.alphabet_generator()
next(alpha_gen) # skipping 'a' as it has already been used above
part_id = 2 # Start part index for pins
for i in range(NX):
    for j in range(NY):
        pcor = ar([LB+i*EH, LB+j*EH, 0]) # Point coordinate for hole / pins
        slab = next(alpha_gen) # Surface label, same for each mating pin so its easeir to track
        # Creating pin
        system_data["PARTS"][str(part_id)] = {}
        system_data["PARTS"][str(part_id)][slab] = {
            "FRAME": Frame1, # Frame doesn't really matter, as long as x is aligned on the axis
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dint / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P0{slab}", f"P1{slab}"], 
            "SURFACE_DIRECTION": "centrifugal",
            "CONSTRAINTS_D": ["PERFECT"], # No defects on the pins
            "BLOCK_ROTATIONS_G": 'x', # The pins do not rotate around their axis
            "BLOCK_TRANSLATIONS_G": 'x', # The pins do not slide along their axis
        }
        # Adding hole to part 0
        system_data["PARTS"]["0"][slab] = {
            "FRAME": Frame1,
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dext / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P{part_id}{slab}"], 
            "SURFACE_DIRECTION": "centripetal",
        }
        # Adding hole to part 1
        system_data["PARTS"]["1"][slab] = {
            "FRAME": Frame2,
            "ORIGIN": pcor, 
            "TYPE": "cylinder",
            "RADIUS": Dext / 2,
            "EXTENT_LOCAL": {"x_max": hPin/2, "x_min": -hPin/2},
            "INTERACTIONS": [f"P{part_id}{slab}"],
            "SURFACE_DIRECTION": "centripetal",
        }
        # Construct Compatibility loop
        loop_id = f"L{part_id-1}"
        formater = lambda i,l : f"P{i}{l}{l.upper()}0" 
        system_data["LOOPS"]["COMPATIBILITY"][loop_id] = f"P0aA0 -> {formater(0,slab)} -> {formater(part_id,slab)} -> {formater(1,slab)} -> P1aA0"
        part_id += 1  

In [6]:
SDA = otaf.AssemblyDataProcessor(system_data)
SDA.generate_expanded_loops()

In [7]:
CLH = otaf.CompatibilityLoopHandling(SDA)
compatibility_expressions = CLH.get_compatibility_expression_from_FO_matrices()

In [8]:
ILH = otaf.InterfaceLoopHandling(SDA, CLH, circle_resolution=CIRCLE_RESOLUTION)
interface_constraints = ILH.get_interface_loop_expressions()

Processing part 0, surface b for cylinder-to-cylinder interactions.
usedGMatDat [['0', 'b', 'B0', '2', 'b', 'B0']]
Found 1 used gap matrices.
unusedGMatDat [['0', 'b', 'B1', '2', 'b', 'B1'], ['0', 'b', 'B2', '2', 'b', 'B2']]
Found 2 unused gap matrices.
Matching used and unused gap matrices: GP0bB0P2bB0 with GP0bB1P2bB1
Matching used and unused gap matrices: GP0bB0P2bB0 with GP0bB2P2bB2
Generated 32 interaction equations for current matching.
Total interaction equations generated: 32
Processing part 0, surface c for cylinder-to-cylinder interactions.
usedGMatDat [['0', 'c', 'C0', '3', 'c', 'C0']]
Found 1 used gap matrices.
unusedGMatDat [['0', 'c', 'C2', '3', 'c', 'C2'], ['0', 'c', 'C1', '3', 'c', 'C1']]
Found 2 unused gap matrices.
Matching used and unused gap matrices: GP0cC0P3cC0 with GP0cC2P3cC2
Matching used and unused gap matrices: GP0cC0P3cC0 with GP0cC1P3cC1
Generated 32 interaction equations for current matching.
Total interaction equations generated: 32
Processing part 0, sur

In [9]:
SOCAM = otaf.SystemOfConstraintsAssemblyModel(
    compatibility_expressions, interface_constraints
)

SOCAM.embedOptimizationVariable()

print(len(SOCAM.deviation_symbols), SOCAM.deviation_symbols)

32 [v_d_0, w_d_0, beta_d_0, gamma_d_0, v_d_2, w_d_2, beta_d_2, gamma_d_2, v_d_5, w_d_5, beta_d_5, gamma_d_5, v_d_7, w_d_7, beta_d_7, gamma_d_7, v_d_8, w_d_8, beta_d_8, gamma_d_8, v_d_10, w_d_10, beta_d_10, gamma_d_10, v_d_11, w_d_11, beta_d_11, gamma_d_11, v_d_13, w_d_13, beta_d_13, gamma_d_13]


## Construction of the stochastic model of the defects.

These are the max variances. 

In [10]:
tol = 0.1 * np.sqrt(2)
Cm = 1  # Process capability

# Defining the uncertainties on the position and orientation uncertainties.
sigma_e_pos = tol / (6 * Cm)
theta_max = tol / hPlate
sigma_e_theta = (2 * theta_max) / (6 * Cm)

In [11]:
RandDeviationVect = otaf.distribution.get_composed_normal_defect_distribution(
    defect_names=SOCAM.deviation_symbols,
    sigma_dict = {"alpha":sigma_e_theta, 
                  "beta":sigma_e_theta,
                  "gamma":sigma_e_theta, 
                  "u":sigma_e_pos, 
                  "v":sigma_e_pos, 
                  "w":sigma_e_pos})
dim_devs = int(RandDeviationVect.getDimension())

## Estimating the bounds on the probability of failure of the model.

- First by exploring the whle parameter space, by generating a LHS in the (normalized) parameter space and doing a double loop montecarlo
- By using the parameter constraint function to guide an optimization algorithm to find the bounds on the probability of failure

## Now lets first to a double loop monte-carlo to explore the full space of the parameters (using the intermediate lambda space) and the stochastic space, to be able to draw the full P-Box of the slack

### Explicit constraint function on the 4 degrees of freedom of the cylindrical tolerance zone :

$$λu2​+λv2​+λα2​+λβ2​+2(λu​λα​+λv​λβ​)−1=0$$

This the constraint on the multipliers of the max standard deviation for each component of the DOFs. The defects are normal, centered and independent

In [12]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
bounds = None
SEED_MC_PF = 6436431
SIZE_MC_PF = 20000 #int(1e6) #1e4
s_values = np.zeros((N_lambda, SIZE_MC_PF)) # For each MC point we have a s value
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.
failure_probabilities = []
# Generalized lambda distribution object for fitting
gld = GLD('VSL')

start_time = time()  # Record the start time
for i in range(N_lambda):
    print(f"Doing iteration {i} of {N_lambda}")
    if i>0:
        fp = failure_probabilities
        print(f"Failure probability i-1 : {fp[i-1]}, Min: {min(fp)}, / Max: {max(fp)}")
        print("s_mean: ", s_values.mean().round(3), "s_min: ", np.nanmin(s_values).round(3), "s_max: ", np.nanmax(s_values).round(3))

    ot.RandomGenerator.SetSeed(SEED_MC_PF)
    deviation_samples = np.array(RandDeviationVect.getSample(SIZE_MC_PF)) * np.array(
        lambda_sample_conditioned[i]
    )
    optimizations = otaf.uncertainty.compute_gap_optimizations_on_sample(
            SOCAM,
            deviation_samples,
            bounds=bounds,
            n_cpu=-1,
            progress_bar=True,
        )
    
    slack = np.array([opt.fun for opt in optimizations], dtype=float)*-1 #Normally there aren"t any nans.
    s_values[i,:] = slack
    GLD_parameters.append(gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False))
    failure_probabilities.append(gld.CDF_num(0, GLD_parameters[i]))


print(f"Done {len (lambda_sample_conditioned)} experiments.")
print(f"Elapsed time: {time() - start_time:.3f} seconds.")

In [ ]:
X = otaf.sampling.find_best_worst_quantile(np.array(lambda_sample_conditioned), np.array(failure_probabilities), 0.1)
(best_5p_lambda, best_5p_res), (worst_5p_lambda, worst_5p_res) = X

In [ ]:
distributions = [ot.UserDefined(ot.Sample(s[:,np.newaxis])) for s in s_values]
x_min, x_max = -0.03, 0.11
sup_data, inf_data = otaf.distribution.compute_sup_inf_distributions(distributions, x_min, x_max)

In [ ]:
# Initialize colors and legends for each lambda sample
colors = [["grey"]] * lambda_sample_conditioned.getSize()
legends = [""] * lambda_sample_conditioned.getSize()

# Plot the combined CDF with additional curves for the envelopes
graph_full = otaf.plotting.plot_combined_CDF(distributions, x_min, x_max, colors, legends)
graph_full = otaf.plotting.set_graph_legends(
    graph_full,
    x_title="j",
    y_title="P",
    title="Slack P-BOX 32 dimensions",
    legends=legends
)

# Add the upper and lower envelopes to the graph
graph_full.add(ot.Curve(inf_data, "blue", "solid", 1.5, "lower envelope"))
graph_full.add(ot.Curve(sup_data, "red", "solid", 1.5, "upper envelope"))
view = viewer.View(graph_full, pixelsize=(1100, 750))

In [ ]:
min(failure_probabilities)

# Now we'll use the other lambda modelization, a bit more difficutl to sample

In [ ]:
lambda_limit_sample = otaf.sampling.generate_imprecise_probabilistic_samples([4]*8, -1, discretization=1)
lambda_limit_sample_sub_choice = [next(lambda_limit_sample) for _ in range(60000)]
lambda_limit_sample_sub_choice = np.stack(lambda_limit_sample_sub_choice)
print(lambda_limit_sample_sub_choice.shape)

In [ ]:
def solve_for_lambda_beta(lambda_u, lambda_v, lambda_alpha):
    """
    Solve the equation for lambda_beta given lambda_u, lambda_v, and lambda_alpha.
    """
    def equation(lambda_beta):
        return (lambda_u**2 + lambda_v**2 + lambda_alpha**2 + lambda_beta**2 +
                2 * (lambda_u * lambda_alpha + lambda_v * lambda_beta) - 1)
    
    # Use a root-finding method to solve for lambda_beta
    lambda_beta_initial_guess = 0.5
    lambda_beta_solution = fsolve(equation, lambda_beta_initial_guess)[0]
    
    return lambda_beta_solution

def generate_points_on_surface(num_points=1000):
    """
    Generate points on the surface by solving for lambda_beta
    given random values for lambda_u, lambda_v, and lambda_alpha.
    
    Returns:
    - A numpy array of shape (num_valid_points, 4) where each row is 
      (lambda_u, lambda_v, lambda_alpha, lambda_beta).
    """
    # Generate random values for lambda_u, lambda_v, lambda_alpha in [0, 1]
    lambda_u = np.random.rand(num_points)
    lambda_v = np.random.rand(num_points)
    lambda_alpha = np.random.rand(num_points)
    
    # Assume lambda_beta = 0 and check if the inequality is satisfied
    inequality_values = lambda_u**2 + lambda_v**2 + lambda_alpha**2 + 2 * (lambda_u * lambda_alpha)

    # Filter valid points where inequality holds
    valid_mask = inequality_values <= 1
    lambda_u_valid = lambda_u[valid_mask]
    lambda_v_valid = lambda_v[valid_mask]
    lambda_alpha_valid = lambda_alpha[valid_mask]
    
    # Solve for lambda_beta for valid points
    lambda_beta_valid = np.array([solve_for_lambda_beta(u, v, a) 
                                  for u, v, a in zip(lambda_u_valid, lambda_v_valid, lambda_alpha_valid)])

    # Only keep points where the solved lambda_beta is between 0 and 1
    final_mask = (lambda_beta_valid >= 0) & (lambda_beta_valid <= 1)
    final_points = np.vstack((lambda_u_valid[final_mask],
                              lambda_v_valid[final_mask],
                              lambda_alpha_valid[final_mask],
                              lambda_beta_valid[final_mask])).T
    
    return final_points

# Generate the points on the surface
points = generate_points_on_surface(num_points=20000)

# Call the pair_plot function (assuming it's already defined)
labels = ['lambda_u', 'lambda_v', 'lambda_alpha', 'lambda_beta']
#otaf.plotting.pair_plot(points, labels)

In [ ]:
random_lambda_sample = [points[np.random.choice(points.shape[0], 200, replace=False)] for _ in range(8)]
random_lambda_sample = np.hstack(random_lambda_sample)

In [ ]:
N_lambda = random_lambda_sample.shape[0]
bounds = None
SEED_MC_PF = 6436431
SIZE_MC_PF = 10000 #int(1e6) #1e4
s_values = np.zeros((N_lambda, SIZE_MC_PF)) # For each MC point we have a s value
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.
failure_probabilities = []
# Generalized lambda distribution object for fitting
gld = GLD('VSL')

start_time = time()  # Record the start time
for i in range(N_lambda):
    print(f"Doing iteration {i} of {N_lambda}")
    if i>0:
        fp = failure_probabilities
        print(f"Failure probability i-1 : {fp[i-1]}, Min: {min(fp)}, / Max: {max(fp)}")
        print("s_mean: ", s_values.mean().round(3), "s_min: ", np.nanmin(s_values).round(3), "s_max: ", np.nanmax(s_values).round(3))

    ot.RandomGenerator.SetSeed(SEED_MC_PF)
    deviation_samples = np.array(RandDeviationVect.getSample(SIZE_MC_PF)) * random_lambda_sample[i]
    optimizations = otaf.uncertainty.compute_gap_optimizations_on_sample(
            SOCAM,
            deviation_samples,
            bounds=bounds,
            n_cpu=-1,
            progress_bar=True,
        )
    
    slack = np.array([opt.fun for opt in optimizations], dtype=float)*-1 #Normally there aren"t any nans.
    s_values[i,:] = slack
    GLD_parameters.append(gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False))
    failure_probabilities.append(gld.CDF_num(0, GLD_parameters[i]))


print(f"Done {N_lambda} experiments.")
print(f"Elapsed time: {time() - start_time:.3f} seconds.")

In [ ]:
X = otaf.sampling.find_best_worst_quantile(np.array(lambda_sample_conditioned), np.array(failure_probabilities), 0.1)
(best_5p_lambda, best_5p_res), (worst_5p_lambda, worst_5p_res) = X

In [ ]:
distributions = [ot.UserDefined(ot.Sample(s[:,np.newaxis])) for s in s_values]
x_min, x_max = -0.03, 0.11
sup_data, inf_data = otaf.distribution.compute_sup_inf_distributions(distributions, x_min, x_max)

In [ ]:
%matplotlib inline

In [ ]:
# Initialize colors and legends for each lambda sample
colors = [["grey"]] * N_lambda
legends = [""] * N_lambda

# Plot the combined CDF with additional curves for the envelopes
graph_full = otaf.plotting.plot_combined_CDF(distributions, x_min, x_max, colors, legends)
graph_full = otaf.plotting.set_graph_legends(
    graph_full,
    x_title="slack",
    y_title="$P_{slack}$",
    title="Slack P-BOX 32 dimensions",
    legends=legends
)

# Add the upper and lower envelopes to the graph
graph_full.add(ot.Curve(inf_data, "blue", "solid", 1.5, "lower envelope"))
graph_full.add(ot.Curve(sup_data, "red", "solid", 1.5, "upper envelope"))
view = viewer.View(graph_full, pixelsize=(1100, 750))

In [ ]:
# %matplotlib qt
fig = plt.figure(dpi=150)

ax = fig.add_subplot(1, 1, 1)
ax.plot(sup_data[:, 0], sup_data[:, 1], color="tab:orange", label="upper envelope")
ax.plot(inf_data[:, 0], inf_data[:, 1], color="tab:blue", label="lower envelope")
ax.grid(True)
ax.fill_between(
    inf_data[:, 0], inf_data[:, 1], sup_data[:, 1], color="gray", alpha=0.3
)
ax.set_xlabel("slack")
ax.set_ylabel("$P_{slack}$")
ax.legend()
ax.set_title("Slack P-Box 3D Assembly 32D")

In [ ]:
print(f"Minimum failure probability: {np.min(failure_probabilities):.5e}, Maximum failure probability: {np.max(failure_probabilities):.5e}")

In [ ]:
STOP

## Global optimization basinhopping with GLD

### First we define the optimization functions 

In [13]:
SIZE_MC_PF = 10000 #int(1e6) #1e4
sample_gld = np.array(RandDeviationVect.getSample(SIZE_MC_PF))
scale_factor = 1.0
GLD_parameters = [] # We need the parameters of the generalized lambda distribution.

# Generalized lambda distribution object for fitting
gld = GLD('VSL')

def model_base(x, sample=sample_gld):
    # Model without surrogate, to get slack
    x = sample * np.sqrt(x[np.newaxis, :])
    optimization_variables = otaf.uncertainty.compute_gap_optimizations_on_sample_batch(
        constraint_matrix_generator=SOCAM,
        deviation_array=x,
        batch_size=500,
        n_cpu=-1,
        progress_bar=True,
        verbose=0,
        dtype="float32",
    )
    slack_values = optimization_variables[:,-1]
    return slack_values

@otaf.optimization.scaling(scale_factor)
def optimization_function_mini(x, model=model_base):
    # Here we search the minimal probability of failure
    slack = model(x)
    gld_params = gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False)
    if np.any(np.isnan(gld_params)):
        failure_probability = np.where(slack<0,1,0).mean()
    else :
        print("gld_params:", gld_params)
        failure_probability = gld.CDF_num(0, gld_params)
    return failure_probability

@otaf.optimization.scaling(scale_factor)
def optimization_function_maxi(x, model=model_base):
    # Here we search the maximal probability of failure so negative output
    slack = model(x)
    gld_params = gld.fit_LMM(slack, disp_fit=False, disp_optimizer=False)
    if np.any(np.isnan(gld_params)):
        failure_probability = np.where(slack<0,1,0).mean()
    else :
        print("gld_params:", gld_params)
        failure_probability = gld.CDF_num(0, gld_params)
    return failure_probability*-1

### Explicit parameter feature constraints (not ideal)

In [14]:
def cylinder_feature_constraint(la):
    """Evaluates the cylinder constraint for a given lambda array."""
    sum_squares = np.dot(la, la)  # Equivalent to la[0]**2 + la[1]**2 + la[2]**2 + la[3]**2
    cross_terms = 2 * (la[0] * la[2] + la[1] * la[3])
    return sum_squares + cross_terms - 1

def cylinder_feature_constraint_jacobian(la):
    """Computes the analytical Jacobian of the cylinder feature constraint."""
    u, v, alpha, beta = la
    return np.array([
        2 * u + 2 * alpha,  # df/du
        2 * v + 2 * beta,   # df/dv
        2 * alpha + 2 * u,  # df/dalpha
        2 * beta + 2 * v    # df/dbeta
    ])

def assembly_feature_constraint(la8):
    """Applies the cylinder constraint to each 4-element slice of the lambda array, returning a vector of constraints."""
    return np.array([cylinder_feature_constraint(la8[i : i + 4]) for i in range(0, len(la8), 4)])

def assembly_feature_constraint_jacobian(la8):
    """Constructs the Jacobian matrix for the vector of constraints, with each row representing the Jacobian of one cylinder constraint."""
    # Initialize a list to hold each constraint's Jacobian as a row in the matrix
    jacobian = []
    # Compute Jacobian for each 4-element segment in the lambda array
    for i in range(0, len(la8), 4):
        row_jacobian = np.zeros(len(la8))  # Create a zeroed row of the appropriate length
        row_jacobian[i:i+4] = cylinder_feature_constraint_jacobian(la8[i:i+4])
        jacobian.append(row_jacobian)
    return np.array(jacobian)


def cylinder_feature_constraint_hessian(la):
    """Computes the Hessian of the cylinder constraint for a given 4-element lambda array."""
    # Extract variables for readability
    u, v, alpha, beta = la
    # The Hessian matrix is symmetric, so we only need to define the unique elements
    hessian = np.array([
        [2, 0, 2, 0],  # d^2f/du^2, d^2f/du dv, d^2f/du dalpha, d^2f/du dbeta
        [0, 2, 0, 2],  # d^2f/dv du, d^2f/dv^2, d^2f/dv dalpha, d^2f/dv dbeta
        [2, 0, 2, 0],  # d^2f/dalpha du, d^2f/dalpha dv, d^2f/dalpha^2, d^2f/dalpha dbeta
        [0, 2, 0, 2]   # d^2f/dbeta du, d^2f/dbeta dv, d^2f/dbeta dalpha, d^2f/dbeta^2
    ])
    return hessian

def assembly_feature_constraint_hessian(la8):
    """Constructs the full Hessian matrix for the vector of constraints in the assembly."""
    n_constraints = len(la8) // 4
    hessian_blocks = []
    
    # Build the Hessian by iterating over each 4-element segment
    for i in range(n_constraints):
        block_hessian = cylinder_feature_constraint_hessian(la8[i*4:i*4+4])
        # Place the block into the full Hessian matrix for the constraint function
        block_hessian_expanded = np.zeros((len(la8), len(la8)))
        block_hessian_expanded[i*4:i*4+4, i*4:i*4+4] = block_hessian
        hessian_blocks.append(block_hessian_expanded)
    
    # Sum the block Hessians to form the full Hessian matrix
    full_hessian = np.sum(hessian_blocks, axis=0)
    return full_hessian

### Implicit parameter constrants (new version)

In [15]:
midof_funcs = otaf.tolerances.MiSdofToleranceZones()

feature_constraint_list = []

# We know that all features are cylindrical, with same values/dimensions
for i in range(8):
    fconst = otaf.tolerances.FeatureLevelStatisticalConstraint(
        midof_funcs.cylindrical_zone,
        mif_args = (tol, hPlate),
        n_dof = 4,
        n_sample = 100000,
        target = "std", #"prob",
        target_val = sigma_e_pos*np.sqrt(1-(2/np.pi)), #0.002699, #
        isNormal = True,
        normalizeOutput = True,
    )
    feature_constraint_list.append(fconst)

composed_assembly_constraint = otaf.tolerances.ComposedAssemblyLevelStatisticalConstraint(feature_constraint_list)

In [16]:
param_bounds_one_feature = [[0.0,0.0], [1e-8, sigma_e_pos], #u, mean std
                            [0.0,0.0], [1e-8, sigma_e_pos], #v, mean std
                            [0.0,0.0], [1e-8, sigma_e_theta], #alpha, mean std
                            [0.0,0.0], [1e-8, sigma_e_theta] # beta, mean std
                           ]
param_bounds = [param_bounds_one_feature] * 8 #We have 8 identical features

normalized_assembly_constraint = otaf.tolerances.NormalizedAssemblyLevelConstraint(
    composed_assembly_constraint,
    param_val_bounds=param_bounds)

#### The assembly constraint takes as an input the list of list of paramters for all the features. But as the distriutions (normals) are suppposed to be centered, we only need to have the standard deviations as inputs so we construct a little intermediary class that takes as an input only the standard deviations and not the means, and completes the means with 0 to pass it to the assembly constraint. 

In [17]:
def assembly_constraint_no_mean(param_list, scale_out=True):
    """ The functions takes directly the concatenated list of all parameters, and reconstructs the right object.
    There should be 32 variables
    """
    assert len(param_list)==32, "problem with input."
    zer = np.zeros(4)
    pl = np.array(param_list)
    params_for_assembly = []
    for i in range(8):
        params = param_list[i*4:i*4+4]
        pa = [item for pair in zip(zer, params) for item in pair]
        params_for_assembly.append(pa)
    res =  normalized_assembly_constraint(params_for_assembly)
    if scale_out :
        return res * 10
    else :
        return res

In [18]:
assembly_constraint_no_mean([1.0,0.0,0.0,0]*8, True)

array([-0.03185387, -0.02547737, -0.03925778,  0.04439285,  0.01699116,
       -0.00300662,  0.01385772, -0.03288411])

#### Now we create the assembly constraint for the optimization (so a non linear constraint)

In [19]:
# Define the nonlinear constraint with the updated vector-valued function and Jacobian
nonLinearConstraint = NonlinearConstraint(
    fun=assembly_constraint_no_mean,
    lb = -0.05 * np.ones((8,)),
    ub = 0.05 * np.ones((8,)),
    jac = "3-point",
    keep_feasible=True,
)

In [20]:
# Initial guess
x0_maxi = [0.25] * RandDeviationVect.getDimension()  # Initial guess

# Perform the local optimization using COBYQA directly
res_maxi = minimize(
    optimization_function_maxi, 
    x0_maxi, 
    method="COBYQA", 
    jac=False, 
    bounds=Bounds(0.0, 1.0, keep_feasible=True),
    constraints = nonLinearConstraint,
    options={
        "f_target": -0.2, 
        "maxiter": 1000,
        "maxfev": 4000,
        "feasibility_tol": 1e-2,
        "initial_tr_radius": np.sqrt(32)*0.5,
        "final_tr_radius": 0.33,
        "disp": True
    }
)

print("Maximization Result with COBYQA:")
print(res_maxi)

Starting the optimization procedure.
Initial trust-region radius: 2.8284271247461903.
Final trust-region radius: 0.33.
Maximum number of function evaluations: 4000.
Maximum number of iterations: 1000.



  0%|          | 0/10000 [00:00<?, ?it/s]

wrapper([ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]) = -0.0
assembly_constraint_no_mean([ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]) = [-1.000e+01 -1.000e+01 ... -1.000e+01 -1.000e+01]


/home/ksimady/anaconda3/envs/otaf_test_env/lib/python3.13/site-packages/gldpy/_gld_module.py:1026: RuntimeWarning: invalid value encountered in scalar divide
  return l1,l2,l3/l2,l4/l2
/home/ksimady/anaconda3/envs/otaf_test_env/lib/python3.13/site-packages/gldpy/_gld_module.py:1026: RuntimeWarning: divide by zero encountered in scalar divide
  return l1,l2,l3/l2,l4/l2
/home/ksimady/anaconda3/envs/otaf_test_env/lib/python3.13/site-packages/gldpy/_gld_module.py:1095: RuntimeWarning: invalid value encountered in scalar add
  if a4**2+98*a4 +1 <0:
/home/ksimady/anaconda3/envs/otaf_test_env/lib/python3.13/site-packages/gldpy/_gld_module.py:1097: RuntimeWarning: invalid value encountered in scalar add
  p4 = np.array([(3+7*a4 + np.sqrt(a4**2+98*a4 +1))/(2*(1-a4)), (3+7*a4 - np.sqrt(a4**2+98*a4 +1))/(2*(1-a4))])


  0%|          | 0/10000 [00:00<?, ?it/s]

...

gld_params: [0.04387239 0.02400161 0.35241898 0.13668658]
wrapper([ 0.000e+00  1.025e-19 ...  1.000e+00  8.411e-01]) = -0.02756776639720525
assembly_constraint_no_mean([ 0.000e+00  1.025e-19 ...  1.000e+00  8.411e-01]) = [ 8.725e-02  4.313e-02 ...  1.286e-01  6.545e-02]


  0%|          | 0/10000 [00:00<?, ?it/s]

gld_params: [0.04369234 0.02398952 0.3531425  0.13595925]
wrapper([ 4.416e-18  4.922e-22 ...  1.000e+00  8.274e-01]) = -0.028026051724631282
assembly_constraint_no_mean([ 4.416e-18  4.922e-22 ...  1.000e+00  8.274e-01]) = [ 1.154e-01  8.775e-02 ...  1.125e-02  5.285e-02]


  0%|          | 0/10000 [00:00<?, ?it/s]

gld_params: [0.0447936  0.02370219 0.34696385 0.13468662]
wrapper([ 0.000e+00  0.000e+00 ...  1.000e+00  8.118e-01]) = -0.024990492663681047
assembly_constraint_no_mean([ 0.000e+00  0.000e+00 ...  1.000e+00  8.118e-01]) = [ 1.599e-01 -1.559e-01 ...  4.846e-01 -4.901e-02]

The lower bound for the trust-region radius has been reached.
Number of function evaluations: 200.
Number of iterations: 86.
Least value of wrapper: -0.02756776639720525.
Maximum constraint violation: 0.07862402818778157.
Corresponding point: [ 0.000e+00  1.025e-19 ...  1.000e+00  8.411e-01].
Maximization Result with COBYQA:
 message: The lower bound for the trust-region radius has been reached
 success: False
  status: 0
     fun: -0.02756776639720525
       x: [ 0.000e+00  1.025e-19 ...  1.000e+00  8.411e-01]
     nit: 86
   maxcv: 0.07862402818778157
    nfev: 200


In [ ]:
# Basinhopping for the maximization function using COBYQA
x0_maxi = [0.25] * RandDeviationVect.getDimension()  # Initial guess

# Update minimizer_kwargs_maxi to use COBYQA
minimizer_kwargs_maxi = {
    "method": "COBYQA",   # Use COBYQA method
    "jac": False,         # COBYQA doesn't use Jacobians
    "args": (model_base,),     # Update args to match COBYQA requirements
    "constraints": nonLinearConstraint,
    "bounds": Bounds(lb=0.0,ub=1.0),
    "options": {
        "f_target": -0.2, 
        "maxiter": 1000,
        "maxfev": 4000,
        "feasibility_tol": 1e-2,
        "initial_tr_radius": np.sqrt(32)*0.5,
        "final_tr_radius": 0.33,
        "disp": True
    }
}

# Running basinhopping with COBYQA as the local optimizer
res_maxi = basinhopping(
    optimization_function_maxi, x0_maxi,
    niter=5,
    T=1,
    stepsize=3.0,
    niter_success=19,
    interval=5,
    minimizer_kwargs=minimizer_kwargs_maxi,
    disp=True,
    #take_step=step_taking,
    #accept_test=accept_test,
    #callback=callback
)

print("Maximization Result with COBYQA:")
print(res_maxi)

In [ ]:
res_maxi.x

In [ ]:
optimization_function_maxi(np.array([0,0,1,0]*8))

In [ ]:
N = ot.Normal()
func = ot.SymbolicFunction(["x"], ["abs(x)"])